In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
model_id = "tiiuae/falcon-mamba-7b"
#model_id = "meta-llama/Llama-2-7b-hf"
#model_id = "openai-community/gpt2"
torch_dtype = torch.bfloat16

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, use_cache=False).to("mps")
print(model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
input_sequence = tokenizer.encode("the largest crater on the moon is", return_tensors="pt", add_special_tokens=False).to("mps")
print(input_sequence)

In [ ]:
model_output = model.generate(input_sequence, max_new_tokens=10, return_dict_in_generate=True, output_hidden_states=True, do_sample=False)
print(tokenizer.decode(model_output["sequences"][0]))

In [ ]:
hidden_states_mps = model_output["hidden_states"]

In [ ]:
del model
del model_output
torch.mps.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch_dtype, use_cache=False)
print(model)

In [ ]:
input_sequence = input_sequence.to("cpu")

In [ ]:
model_output = model.generate(input_sequence, max_new_tokens=10, return_dict_in_generate=True, output_hidden_states=True, do_sample=False)
print(tokenizer.decode(model_output["sequences"][0]))

In [ ]:
hidden_states_cpu = model_output["hidden_states"]

In [ ]:
del model
del model_output

In [ ]:
# for token, (hidden_state_mps, hidden_state_cpu) in enumerate(zip(hidden_states_mps, hidden_states_cpu)):
#     print(f"Token {token}:")
#     for layer in range(len(hidden_state_mps)):
#         print((hidden_state_mps[layer].to("cpu") - hidden_state_cpu[layer]).abs().max())
for layer in range(len(hidden_states_mps[0])):
    print(layer, (hidden_states_mps[0][layer].to("cpu") - hidden_states_cpu[0][layer]).abs().max())